In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

# Find the project root (the nearest ancestor that contains `src`) and add it to sys.path
cwd = Path.cwd()
project_root = cwd
for _ in range(6):
    if (project_root / "src").exists():
        break
    if project_root.parent == project_root:
        break
    project_root = project_root.parent
else:
    project_root = cwd

proj_path = str(project_root.resolve())
if proj_path not in sys.path:
    # Insert at front so local packages take precedence
    sys.path.insert(0, proj_path)

import aerosandbox.numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from src.airfoil.compute_airfoil_quality import compute_airfoil_quality, QualityError

import aerosandbox as asb
from src.airfoil import airfoil_modifications

import plotly.express as px

## Extraction

In [2]:
airfoil_database_path = asb._asb_root / "geometry" / "airfoil" / "airfoil_database"

airfoil_database = [
    asb.Airfoil(name=filename.stem).normalize()
    for filename in airfoil_database_path.glob("*.dat")
]

## Preprocessing

### Quality check

In [3]:
# Remove airfoils that don't pass the quality tests
quality_airfoil_database = airfoil_database.copy()

for af in airfoil_database:
    try:
        compute_airfoil_quality(af, airfoil_database_path)
    except QualityError as e:
        quality_airfoil_database.remove(af)
        print(f"Airfoil {af.name.ljust(20)} failed quality checks: {e}")
        # af.draw()

Airfoil fx79w470a            failed quality checks: Airfoil has abnormally large changes in angle at (0.977417, 0.0876827), 21.6 deg.
Airfoil mh112                failed quality checks: Airfoil has abnormally high x-coordinates.
Airfoil as6095               failed quality checks: Airfoil has negative thickness.
Airfoil fx79w660a            failed quality checks: Airfoil has abnormally large changes in angle at (0.948442, -0.203101), 34.7 deg.


In [4]:
from aerosandbox.geometry.airfoil.airfoil_families import get_kulfan_parameters

LW_CAP = 8.0
UW_CAP = 8.0
THICKNESS_CAP = 0.01
LE_CAP = 2

def build_dataset(airfoils_database: list):
  airfoils_data = []

  for airfoil in airfoils_database:
    try:
      parameters = get_kulfan_parameters(
          coordinates=airfoil.coordinates,
          n_weights_per_side=12,
          normalize_coordinates=False,
          use_leading_edge_modification=False,
      )

      # Remove outliers
      _lw = parameters["lower_weights"]
      _uw = parameters["upper_weights"]
      _te = parameters["TE_thickness"]
      _le = parameters["leading_edge_weight"]
      if any(_lw > LW_CAP) or any(_uw > UW_CAP) or (_te > THICKNESS_CAP) or (-1 * LE_CAP > _le or _le > LE_CAP):
        continue

      airfoils_data.append([airfoil.name, airfoil.coordinates, parameters])
    except:
      # airfoils_data.append([airfoil.name, airfoil.coordinates, None])
      pass

  airfoil_dataset = pd.DataFrame(airfoils_data, columns=["airfoil_name", "coordinates", "kulfan_parameters"])
  airfoil_dataset["shape"] = airfoil_dataset.coordinates.apply(lambda coords: coords.shape)
  airfoil_dataset["points"] = airfoil_dataset["shape"].apply(lambda shape: shape[0])

  return airfoil_dataset

airfoil_dataset = build_dataset(quality_airfoil_database)
display(airfoil_dataset)

,airfoil_name,coordinates,kulfan_parameters,shape,points
0,p51droot,"[[1.0, 0.0], [0.977786, 0.002797], [0.950485, ...","{'lower_weights': [-0.17226500384533514, -0.07...","(41, 2)",41
1,mid104,"[[0.999998928536621, 0.0025210449234366853], [...","{'lower_weights': [-0.12002044446629195, -0.16...","(200, 2)",200
2,mid117,"[[1.0000079372691175, 0.0038361331869844893], ...","{'lower_weights': [-0.11092218400817529, -0.10...","(140, 2)",140
3,hn979d,"[[1.0, 0.0], [0.999013, 0.000124], [0.996057, ...","{'lower_weights': [-0.07874005028524055, -0.13...","(101, 2)",101
4,s4022,"[[1.0, 0.0], [0.99679, 0.00128], [0.98795, 0.0...","{'lower_weights': [-0.12710624050453254, -0.27...","(62, 2)",62
...,...,...,...,...,...
2019,sc20710,"[[0.99996484973823, 0.002449495593743594], [0....","{'lower_weights': [-0.16229282680035162, -0.03...","(205, 2)",205
2020,hq358,"[[1.0, 0.0], [0.975, 0.00494], [0.95, 0.01008]...","{'lower_weights': [-0.05231176213201487, -0.08...","(77, 2)",77
2021,cb2515,"[[1.0, 0.0], [0.99, 0.00269], [0.95, 0.01093],...","{'lower_weights': [-0.18835680916811873, -0.32...","(43, 2)",43
2022,lds2,"[[1.0, 0.0], [0.9502, 0.008912], [0.9005, 0.01...","{'lower_weights': [-0.12502348089197102, -0.02...","(57, 2)",57


### Second Quality Check


In [5]:
airfoil_dataset["lower_weights"] = airfoil_dataset["kulfan_parameters"].apply(lambda params: params["lower_weights"])
airfoil_dataset["upper_weights"] = airfoil_dataset["kulfan_parameters"].apply(lambda params: params["upper_weights"])
airfoil_dataset["TE_thickness"] = airfoil_dataset["kulfan_parameters"].apply(lambda params: params["TE_thickness"])
airfoil_dataset["leading_edge_weight"] = airfoil_dataset["kulfan_parameters"].apply(lambda params: params["leading_edge_weight"])

lw = airfoil_dataset["lower_weights"].to_numpy()
lw = np.vstack(lw)
lw = lw.flatten()

le = airfoil_dataset["leading_edge_weight"].to_numpy()
le = np.vstack(le)
le = le.flatten()

te = airfoil_dataset["TE_thickness"].to_numpy()
te = np.vstack(te)
te = te.flatten()

# Print a boxplot to visualize the distribution of lower_weights
fig = px.box(
    pd.DataFrame(lw),
    points="all",
    title="Distribution of Lower Weights",
)
fig.show()

fig = px.box(
    pd.DataFrame(le),
    points="all",
    title="Distribution of Leading Edge Weights",
)
fig.show()

fig = px.box(
    pd.DataFrame(te),
    points="all",
    title="Distribution of TE Thickness",
)
fig.show()

In [8]:
a = airfoil_dataset.sort_values(by="leading_edge_weight", inplace=False, ascending=True)
print(a["leading_edge_weight"].iloc[0])
asb.Airfoil(coordinates=a["coordinates"].iloc[0]).draw()

-1.972427240387408


In [9]:
fig = px.histogram(airfoil_dataset, x="points", title="Total number of points distribution")
fig.show()

### Coordinates standardization

In [10]:
# The number of coordinates for each airfoil is inconsistent across the database, so we use Cubic splines interpolation to standadize the coordinates
n_points_per_side = 75
normalized_coords = [asb.Airfoil(coordinates=coords).repanel(n_points_per_side).coordinates for coords in airfoil_dataset["coordinates"]]

std_airfoil_dataset = airfoil_dataset.copy()
std_airfoil_dataset["coordinates"] = normalized_coords
std_airfoil_dataset["shape"] = std_airfoil_dataset.coordinates.apply(lambda coords: coords.shape)
std_airfoil_dataset["points"] = std_airfoil_dataset["shape"].apply(lambda shape: shape[0])
display(std_airfoil_dataset)

,airfoil_name,coordinates,kulfan_parameters,shape,points,lower_weights,upper_weights,TE_thickness,leading_edge_weight
0,p51droot,"[[1.0, 0.0], [0.9995378454897893, 5.7056950520...","{'lower_weights': [-0.17226500384533514, -0.07...","(149, 2)",149,"[-0.17226500384533514, -0.07285531046477561, -...","[0.2159684003495692, 0.3039354287094531, 0.095...",0.000000e+00,-0.203910
1,mid104,"[[0.999998928536621, 0.0025210449234366853], [...","{'lower_weights': [-0.12002044446629195, -0.16...","(149, 2)",149,"[-0.12002044446629195, -0.16010435997486497, 0...","[0.0967838845554305, 0.20907738969513012, 0.10...",5.012069e-03,0.273667
2,mid117,"[[1.0000079372691175, 0.0038361331869844893], ...","{'lower_weights': [-0.11092218400817529, -0.10...","(149, 2)",149,"[-0.11092218400817529, -0.10030024938082963, 0...","[0.12575160307175953, 0.22552791928936045, 0.1...",7.664298e-03,0.128861
3,hn979d,"[[1.0, 0.0], [0.9995480532265133, 5.6558136145...","{'lower_weights': [-0.07874005028524055, -0.13...","(149, 2)",149,"[-0.07874005028524055, -0.1342005162183766, 0....","[0.07995488337116266, 0.11363895065699736, 0.0...",0.000000e+00,0.166533
4,s4022,"[[1.0, 0.0], [0.9995669891676611, 0.0001711392...","{'lower_weights': [-0.12710624050453254, -0.27...","(149, 2)",149,"[-0.12710624050453254, -0.27746085069476256, 0...","[0.11201029755940403, 0.07899098812456795, 0.3...",0.000000e+00,0.573019
...,...,...,...,...,...,...,...,...,...
2019,sc20710,"[[0.99996484973823, 0.002449495593743594], [0....","{'lower_weights': [-0.16229282680035162, -0.03...","(149, 2)",149,"[-0.16229282680035162, -0.030155541181530854, ...","[0.1747688853991338, 0.07168351183416768, 0.27...",4.950936e-03,-0.050145
2020,hq358,"[[1.0, 0.0], [0.9995480212585446, 8.8258462866...","{'lower_weights': [-0.05231176213201487, -0.08...","(149, 2)",149,"[-0.05231176213201487, -0.08689953593982726, 0...","[0.1202790033684423, 0.21052189137402808, 0.10...",0.000000e+00,0.109163
2021,cb2515,"[[1.0, 0.0], [0.9995490802498828, 0.0001246888...","{'lower_weights': [-0.18835680916811873, -0.32...","(149, 2)",149,"[-0.18835680916811873, -0.3285910869067207, 0....","[0.15118922507692917, 0.27293687323460303, 0.0...",1.733332e-06,0.357708
2022,lds2,"[[1.0, 0.0], [0.9995438711260742, 8.2228603793...","{'lower_weights': [-0.12502348089197102, -0.02...","(149, 2)",149,"[-0.12502348089197102, -0.029515660379299397, ...","[0.21130914876687804, 0.205784257688015, 0.165...",9.040569e-07,-0.041119


In [11]:
std_airfoil_dataset["TE_thickness"] = std_airfoil_dataset["kulfan_parameters"].apply(lambda params: params["TE_thickness"])
std_airfoil_dataset["leading_edge_weight"] = std_airfoil_dataset["kulfan_parameters"].apply(lambda params: params["leading_edge_weight"])
std_airfoil_dataset[["TE_thickness", "leading_edge_weight"]].describe()

,TE_thickness,leading_edge_weight
count,2024.000000,2024.000000
mean,0.001133,0.152098
std,0.001984,0.521721
min,0.000000,-1.972427
25%,0.000000,-0.008786
50%,0.000002,0.146345
75%,0.001577,0.374835
max,0.009997,1.990688


In [12]:
std_airfoil_dataset["points"].describe()

count    2024.0
mean      149.0
std         0.0
min       149.0
25%       149.0
50%       149.0
75%       149.0
max       149.0
Name: points, dtype: float64

### Saving dataset

In [15]:
validation_std_airfoil_dataset = std_airfoil_dataset.sample(frac=0.1, random_state=42)

# Dropping samples from original to avoid data leakage
train_std_airfoil_dataset = std_airfoil_dataset.drop(validation_std_airfoil_dataset.index)

# Saving both datasets to json files
validation_std_airfoil_dataset.to_json(rf"../../data/processed/val_kulfan_dataset_{n_points_per_side}.json")

train_std_airfoil_dataset.to_json(rf"../../data/processed/train_kulfan_dataset_{n_points_per_side}.json")

print(f"Total: {len(std_airfoil_dataset)}")
print(f"Train: {len(train_std_airfoil_dataset)}")
print(f"Val:   {len(validation_std_airfoil_dataset)}")

Total: 1822
Train: 1640
Val:   182
